In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

The relationship between an interplanar distance and its Miller indices is expressed as follows (Eq. 1):

$$d_{hkl} = \frac{1}{\sqrt{Q_{11} h^2 + Q_{22} k^2 + Q_{33} l^2 + Q_{12} hk + Q_{13} hl + Q_{23} kl}}$$ 

The last equation is valid for all crystal systems. The relationship of each $Q_{ij}$ coefficient and the lattice parameters $a$, $b$, $c$, $\alpha$, $\beta$, and $\gamma$ is defined in the next cell and can be found in International Tables for Crystallography Volume H, Chapter 3.4, pp. 270-281. 

In [ ]:
def dhkl(abc = list(), angles=(), h=1, k=1, l=1):
    """
        A function to compute the equation given above.
        Paramaters:
            abc: list, with the unit cell vectors
            angles: list, with the angles of the unit cells.
            h, k, l are integers and are the Miller index
        Return: a float, which is the interplanar distance associated to the family of planes hkl
    """
    a, b, c = abc[0], abc[1], abc[2]
    alpha, beta, gamma = angles[0], angles[1], angles[2]
    
    A = (a**2)*(b**2)*(c**2)*(1 - np.cos(alpha)**2 - np.cos(beta)**2 - np.cos(gamma)**2 + 2*np.cos(alpha)*np.cos(beta)*np.cos(gamma))
    
    q11 = (b**2)*(c**2)*np.sin(alpha)**2
    q22 = (a**2)*(c**2)*np.sin(beta)**2
    q33 = (a**2)*(b**2)*np.sin(gamma)**2
    
    q12 = 2*a*b*(c**2)*(np.cos(alpha)*np.cos(beta)-np.cos(gamma))
    q13 = 2*a*c*(b**2)*(np.cos(alpha)*np.cos(gamma)-np.cos(beta))
    q23 = 2*b*c*(a**2)*(np.cos(beta)*np.cos(gamma)-np.cos(alpha))
    
    Q11 = q11/A
    Q22 = q22/A
    Q33 = q33/A
    
    Q12 = q12/A
    Q13 = q13/A
    Q23 = q23/A
    
    return (Q11*h**2 + Q22*k**2 + Q33*l**2 + Q12*h*k + Q13*h*l + Q23*k*l)**-0.5
    

The main idea in our approach is that general expression given above can be rewritten as follows (Eq. 2):

$$d_{hkl}^{-2} = h^{2}d_{100}^{-2} + k^{2}d_{010}^{-2} + l^{2}d_{001}^{-2} + hk [d_{110}^{-2} -(d_{100}^{-2} + d_{010}^{-2})] + hl [d_{101}^{-2} -(d_{100}^{-2} + d_{001}^{-2})] + kl [d_{011}^{-2} -(d_{010}^{-2} + d_{001}^{-2})] $$

where $d_{100}$, $d_{010}$, $d_{001}$, $d_{110}$, $d_{101}$, and $d_{011}$ are the six interplanar distances that the Convolutional Neural Networks learn to assess. From these hkl-distances, the lattice parameter can be recalculated. There are two motivations to follow the current approach:
<ol>
 <li> Unit-cell vectors and angles possess diferent dimensionality. </li>
 <li> Beyond the existing symmetry constraints, the interplanar distances are actually observed in a powder diffraction pattern and not lattice parameters. In fact, there is a coincidence between unit-cell vectors and interplanar distances when the coordinate system is orthogonal. </li>
</ol>

Now, the interplanar distances will be computed

### dhkl calculation

In [ ]:
#Next line loads the csv that has information about the lattice parameters from each compound
dlat = pd.read_csv('../DataSets/1In-omdb12k-hr/1In-omdb12k-hr_dftest.csv')
dlat = dlat.rename(columns={'name':'cif'})
dlat = dlat[['cif','a','b','c','alpha','beta','gamma']]

In [ ]:
dlat

The next for-loop actually calculates the dhkl values as follows: first, the distances $d_{100}$, $d_{010}$, and $d_{001}$ are calculated from the lattice parameters. Then, the distances are ordered in a decreasing fashion, which also rearranges the order of the unit-cell vectors and their corresponding angles. Finally, all the distances are calculated, including those previously obtained.

In [ ]:
d = np.zeros((dlat.shape[0],6))
for row in range(dlat.shape[0]):
    
    dcast = np.zeros((1,3))
    abc = dlat.iloc[row,-6:-3].values
    abc = abc.astype(np.float32)
    angles = dlat.iloc[row,-3:].values
    angles = angles.astype(np.float32)
    
    angles = np.deg2rad(angles)
    
    dcast[0,0] = dhkl(abc = abc, angles = angles, h=1, k=0, l=0)
    dcast[0,1] = dhkl(abc = abc, angles = angles, h=0, k=1, l=0)
    dcast[0,2] = dhkl(abc = abc, angles = angles, h=0, k=0, l=1)
    
    #The calculated distances are ordered in a decreasing fashion
    idxsort = np.argsort(dcast[0])
    idxsort = [i for i in reversed(idxsort)]
    #The unit-cell vectors and their corresponding angles are rearranged
    abc = abc[idxsort]
    angles = angles[idxsort]
    #The interplanar distances are calculated in the right sequence.
    d[row,0] = dhkl(abc = abc, angles = angles, h=1, k=0, l=0)
    d[row,1] = dhkl(abc = abc, angles = angles, h=0, k=1, l=0)
    d[row,2] = dhkl(abc = abc, angles = angles, h=0, k=0, l=1)
    
    d[row,3] = dhkl(abc = abc, angles = angles, h=1, k=1, l=0)
    d[row,4] = dhkl(abc = abc, angles = angles, h=1, k=0, l=1)
    d[row,5] = dhkl(abc = abc, angles = angles, h=0, k=1, l=1)
    

The order interplanar distances are saved as yts2_sxv.npy


In [ ]:
np.save('./yts2_sxv', d)